# Text extraction from PDF file

Hey David! Go straight to the last paragraph, there's just one TODO left to do ;)

In [1]:
import os

## PyPDF
Raw text, includes texts from charts for instance.  
Can extract page by page, I don't know how useful that is.

In [17]:
import PyPDF2
from PyPDF2 import PdfReader

reader = PdfReader("pdf_examples/example1.pdf")
text = reader.pages[15].extract_text()
text

'UKRAINE RESPONSE16\nIOMFigure 14: Perceptions of difficulties faced upon returning in Ukraine, by return intentions and total (%), n=1,800 \nPROSPECTIVE RETURNEES\nVery diﬃcult \nSomewhat diﬃcult  \nNeither diﬃcult\nnor easy \nSomewhat easy \nVery easyBasic\nservices  Food\nproducts  Cover HH\nexpenses  Renew\ndocuments   Participate in\npublic aﬀairs  Adequate\naccommodation  Cope with\nsecurity \n1%\n1%\n23%\n75%2%1%\n26%\n71%1%\n13%12%\n33%\n51%1%\n5%\n17%\n77%1%\n6%12%\n25%\n66%2%\n3%\n11%\n84%10%\n2%1%\n3%\n84%\n \n \n \n Basic\nservices  Food\nproducts  Cover HH\nexpenses  Renew\ndocuments   Participate in\npublic aﬀairs  Adequate\naccommodation  Cope with\nsecurity \n15%1%\n39%\n45%15%4%\n43%\n38%2%\n23%9% 4%\n29%\n37%1%\n11%\n28%\n56%3%\n13%6%\n32%\n46%1%\n2%\n8%\n16%\n73%21%\n7%11%\n15%\n46%SHORT-TERM VISITORS\nVery diﬃcult\nSomewhat diﬃcult \nNeither diﬃcult\nnor easy\nSomewhat easy\nVery easy\n \n \n \n Basic\nservices  Food\nproducts  Cover HH\nexpenses  Renew\ndocuments  

In [3]:
print(str(reader.metadata))

{'/CreationDate': "D:20240606151944+03'00'", '/Creator': 'Adobe InDesign 19.0 (Macintosh)', '/ModDate': "D:20240606151946+03'00'", '/Producer': 'Adobe PDF Library 17.0', '/Trapped': '/False'}


In [56]:
reader = PdfReader("dtm_reports/report_Somalia_0.pdf")
#text = reader.pages[1].extract_text()

print(str(reader.metadata))

{'/Producer': 'Adobe PDF Library 17.0', '/CreationDate': "D:20240613150747+03'00'", '/Creator': 'Adobe InDesign 19.0 (Windows)', '/ModDate': "D:20240613150751+03'00'", '/Trapped': '/False', '/Title': 'Report_Somalia_0', '/Country': 'Somalia', '/Year': '2024'}


In [76]:
print(text)

UKRAINE RESPONSE16
IOMFigure 14: Perceptions of difficulties faced upon returning in Ukraine, by return intentions and total (%), n=1,800 
PROSPECTIVE RETURNEES
Very diﬃcult 
Somewhat diﬃcult  
Neither diﬃcult
nor easy 
Somewhat easy 
Very easyBasic
services  Food
products  Cover HH
expenses  Renew
documents   Participate in
public aﬀairs  Adequate
accommodation  Cope with
security 
1%
1%
23%
75%2%1%
26%
71%1%
13%12%
33%
51%1%
5%
17%
77%1%
6%12%
25%
66%2%
3%
11%
84%10%
2%1%
3%
84%
 
 
 
 Basic
services  Food
products  Cover HH
expenses  Renew
documents   Participate in
public aﬀairs  Adequate
accommodation  Cope with
security 
15%1%
39%
45%15%4%
43%
38%2%
23%9% 4%
29%
37%1%
11%
28%
56%3%
13%6%
32%
46%1%
2%
8%
16%
73%21%
7%11%
15%
46%SHORT-TERM VISITORS
Very diﬃcult
Somewhat diﬃcult 
Neither diﬃcult
nor easy
Somewhat easy
Very easy
 
 
 
 Basic
services  Food
products  Cover HH
expenses  Renew
documents   Participate in
public aﬀairs  Adequate
accommodation  Cope with
security 
6%
26%
6

## PyMuPDF 4 LLM

That's even worse, it keeps all the line breaks, tables and charts.

In [7]:
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("pdf_examples/example1.pdf")
md_text

In [12]:
print(md_text)

UKRAINE RESPONSE

### REPUBLIC OF MOLDOVA

##### UKRAINIAN NATIONALS CROSSING BACK TO UKRAINE

JANUARY - MARCH 2024


-----

The opinions expressed in this publication are those of the authors and do not necessarily reflect the views of the International
Organization for Migration (IOM). The designations employed and the presentation of material throughout the report do not imply
expression of any opinion whatsoever on the part of IOM concerning the legal status of any country, territory, city or area, or of its
authorities, or concerning its frontiers or boundaries.

IOM is committed to the principle that humane and orderly migration benefits migrants and society. As an intergovernmental
organization, IOM acts with its partners in the international community to: assist in meeting the operational challenges of migration;
advance understanding of migration issues; encourage social and economic development through migration; and uphold the human
dignity and well-being of migrants.

This 

## Attempt to remove the tables

See https://github.com/jsvine/pdfplumber/issues/242


In [15]:
import pdfplumber

def curves_to_edges(cs):
    """See https://github.com/jsvine/pdfplumber/issues/127"""
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

# Import the PDF.
pdf = pdfplumber.open("pdf_examples/example1.pdf")

# Load the first page.
p = pdf.pages[0]

# Table settings.
ts = {
    "vertical_strategy": "explicit",
    "horizontal_strategy": "explicit",
    "explicit_vertical_lines": curves_to_edges(p.curves + p.edges),
    "explicit_horizontal_lines": curves_to_edges(p.curves + p.edges),
    "intersection_y_tolerance": 10,
}

# Get the bounding boxes of the tables on the page.
bboxes = [table.bbox for table in p.find_tables(table_settings=ts)]

def not_within_bboxes(obj):
    """Check if the object is in any of the table's bbox."""
    def obj_in_bbox(_bbox):
        """See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404"""
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

print("Text outside the tables:")
print(p.filter(not_within_bboxes).extract_text())

KeyError: 'y1'

The function 'find_tables' is useless, what a shame

In [53]:
pdf.pages[6].find_tables()[1].extract()

[[''], [''], [''], ['7%']]

In [63]:
pdf.pages[16].find_tables()[0].extract()

[[None,
  '',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['',
  '',
  '',
  '',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  '',
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  '',
  None,
  None],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '16% 15% 16% 15% 15%\n10% 11%11%\n9%',
  '',
  '',
  '',
  '',
  '']]

In [60]:
print(pdf.pages[16].extract_text())

REPUBLIC OF MOLDOVA
NEEDS, ASSISTANCE RECEIVED, AND CHALLENGES
More than one third of all respondents (35%) stated that they this as an immediate need upon crossing back. Considerably
had no immediate needs at the time of the interview. The share more female respondents reported the need for immediate
of respondents that had no needs was higher among short- personal safety and security than male respondents (23% versus
term visitors than prospective returnees (36% versus 21%). 8%). However, the share of men reporting the need for health
The most common needs reported by short-term visitors services (35%) and medicines (29%) was more than double
included financial support (44%), personal safety and security that of women that expressed the same needs (respectively at
(21%), and healthcare services (31%). The most frequent needs 15% and 9%). This could be attributed to 80 per cent of male
of prospective returnees included financial support (42%), respondents being aged older than 60 year

In [ ]:
# if find_tables worked, we could have used something like:
# table_content = pdf.pages[16].find_tables()[0].extract()
# full_page_text = pdf.pages[16].extract_text()
# full_page_text.replace(table_content, "")

## Chunking

In [4]:
import pymupdf4llm
from langchain.text_splitter import MarkdownTextSplitter

In [89]:
# Get the MD text
md_text = pymupdf4llm.to_markdown("pdf_examples/example1.pdf")  # get markdown for all pages

In [102]:
chunk_size = 256
chunk_overlap = 0

splitter = MarkdownTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

docs = splitter.create_documents([md_text])
text_docs = []
for doc in docs:
    # We only keep chunks that are mostly made out of text
    alpha_ratio = sum([x.isalpha() for x in doc.page_content])/len(doc.page_content)
    if alpha_ratio >= 0.7:
        text_docs.append(doc)

print(f"We kept {len(text_docs)} chunks out of {len(docs)}")

We kept 167 chunks out of 226


In [103]:
# TODO: Save documents from text_docs in the right format

# Script python for DTM reports

In [74]:
import sys
import os

import PyPDF2
from PyPDF2 import PdfReader


def main(PATH=".", verbose=True):
    
    reports_dir = os.path.join(PATH, "dtm_reports")
    if not os.path.isdir(reports_dir):
        raise OSError("Can't find pdf reports folder")
    
    nb_reports = len(os.listdir(reports_dir))
    for i_report, report_fn in enumerate(sorted(os.listdir(reports_dir))):
        print(f"Extracting text from ({i_report+1}/{nb_reports}) {report_fn}")
        reader = PdfReader(os.path.join(reports_dir, report_fn))
        
        # Get the MD text
        md_text = pymupdf4llm.to_markdown(os.path.join(reports_dir, report_fn))  # get markdown for all pages
        
        print("Converted pdf to markdown")
        chunk_size = 256
        chunk_overlap = 0

        splitter = MarkdownTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        docs = splitter.create_documents([md_text])
        text_docs = []
        for doc in docs:
            # We only keep chunks that are mostly made out of text
            alpha_ratio = sum([x.isalpha() for x in doc.page_content])/len(doc.page_content)
            if alpha_ratio >= 0.7:
                text_docs.append(doc)

        print(f"We kept {len(text_docs)} chunks out of {len(docs)}")
        
        # TODO: save the docs somewhere

main()

#TODO: once we move to a python script, use:
"""if __name__ == "__main__":
    main(*sys.argv[1:])
"""



Extracting text from (1/29) report_Ethiopia_0.pdf
Extracting text from (2/29) report_Ethiopia_1.pdf


KeyboardInterrupt: 

## Save metadata on pdf

In [5]:
from webscraper import scrap_DTM_reports

In [6]:
PATH='.'
reports_dir="dtm_reports"
chunks_dir="dtm_chunks"
verbose = True
    
reports_dir = os.path.join(PATH, reports_dir)
chunks_dir = os.path.join(PATH, chunks_dir)


In [51]:
import sys
import os

import requests
from bs4 import BeautifulSoup


def scrap_DTM_reports(reports_dir, countries=None, 
                      verbose=True):
    
    # TODO: scrap country codes automatically
    code_countries = {
        'Ethiopia':63,
        'Somalia':80
    }
    
    if countries is None:
        countries = code_countries.keys()
    
    if not os.path.isdir(reports_dir):
        os.mkdir(reports_dir)
    
    year = 2024
    page = 0
    
    for country in countries:
        if country  in code_countries.keys():
            URL = f"https://dtm.iom.int/reports?f%5B0%5D=published_date%3A{year}&f%5B1%5D=report_country%3A{code_countries[country]}&page={page}"

            response = requests.get(URL)
            soup = BeautifulSoup ( response.content , "html.parser" )

            all_reports = soup.find_all('span',
                                  attrs={
                                      'class' :"file file--mime-application-pdf file--application-pdf"
                                     }
                                  )

            all_links = [report.a.attrs['href'] for report in all_reports]

            for i, link in enumerate(all_links):
                response = requests.get("https://dtm.iom.int/" + link)
                file_path = os.path.join(reports_dir, f"report_{country}_{i}.pdf")
                open(file_path, "wb").write(response.content)
                
                # TODO: check metadata, see if we can add some more
                metadata = {
                    '/Title': f'Report_{country}_{i}',
                    '/Country': country,
                    '/Year': str(year)
                }
                
                add_metadata_to_pdf(file_path, metadata)
                
                if verbose:
                    print(f"Report #{i} saved for {country}")
                
                break
        elif verbose:
            print(f"Unknown reference to country {country} code in URL")

    return reports_dir


In [52]:

import pprint
from PyPDF2 import PdfReader, PdfMerger


def add_metadata_to_pdf(filename, metadata):
    print("Adding metadata to:", filename)
    
    file_in = open(filename, 'ab+')
    pdf_reader = PdfReader(file_in)
    old_metadata = pdf_reader.metadata
    pprint.pprint(old_metadata)

    pdf_merger = PdfMerger()
    pdf_merger.append(file_in)
    pdf_merger.add_metadata(dict(old_metadata,**metadata))
    pdf_merger.write(file_in)

    file_in.close()
# TODO: itérer les pages et s'arrêter quand il n'y a plus de rapport
# TODO: regarder les autres années
# TODO: ajouter les autres pays
# -> On s'en passe parce qu'il y a moins de 20 rapports par page, et qu'il y a qu'une seule année de rapports disponible




In [53]:
scrap_DTM_reports(reports_dir=reports_dir, verbose=verbose)

Adding metadata to: ./dtm_reports/report_Ethiopia_0.pdf
{'/CreationDate': "D:20240614140412+03'00'",
 '/Creator': 'Adobe InDesign 19.4 (Windows)',
 '/ModDate': "D:20240616142134+03'00'",
 '/Producer': 'Adobe PDF Library 17.0',
 '/Trapped': '/False'}
Report #0 saved for Ethiopia
Adding metadata to: ./dtm_reports/report_Somalia_0.pdf
{'/CreationDate': "D:20240613150747+03'00'",
 '/Creator': 'Adobe InDesign 19.0 (Windows)',
 '/ModDate': "D:20240613150751+03'00'",
 '/Producer': 'Adobe PDF Library 17.0',
 '/Trapped': '/False'}
Report #0 saved for Somalia


'./dtm_reports'